In [1]:
# test usblib--------------------------------
import usb
las = usb.core.find(find_all=True)
# las = usb.core.find(find_all=True, idVendor=0x0403, idProduct=0x90D9)
devicelist = []
for item in las:
    devicelist.append(item)
print(devicelist)

[<DEVICE ID 0403:90d9 on Bus 000 Address 001>]


In [2]:
import time
import sys
import os
path_project = "\\".join(os.getcwd().split("\\")[:-2])
# caution: path[0] is reserved for script path (or '' in REPL)
sys.path.insert(1, path_project)

from hardware.laser.laser import LaserControl
from hardware import config as hdc

laser = LaserControl(hdc.LASER_SN)
laser.open()

In [3]:
# test laser on/off ------------------------------
laser.laser_on()
assert laser.is_laser_on()
laser.laser_off()
assert not laser.is_laser_on()
assert laser.get_status() == "Standby"

In [4]:
# test constant mode ---------------------------
laser.set_analog_control_mode("power")
assert laser.get_analog_control_mode() == "power"
laser.set_analog_control_mode("current")
assert laser.get_analog_control_mode() != "power"

In [5]:
# test analog modulation -----------------------
laser.set_analog_modulation("iNt")
assert laser.get_analog_modulation() == "int"
laser.set_analog_modulation("Ext")
assert laser.get_analog_modulation() != "int"

In [6]:
# test diode current ----------------------------------
current_percent = 20.75
laser.set_analog_control_mode("current")
laser.set_diode_current(current_percent, save_memory=False)
laser.laser_on()
time.sleep(10) # allow laser current to settle
cp_read = laser.get_diode_current()/laser.get_max_laser_current()
laser.laser_off()
assert round(current_percent, -1) == round(cp_read*100, -1)

In [7]:
# test CDRH state ---------------------------------
laser.set_CDRH_state(True)
assert laser.get_CDRH_state()
laser.set_CDRH_state(False)
assert laser.get_CDRH_state() == False

In [8]:
# test modulation type cw/modulated -------------------------
laser.set_modulation_state("CW")
assert laser.get_modulation_state() == "cw"
laser.set_modulation_state("Pulsed")
assert laser.get_modulation_state() == "modulated"

In [9]:
# test laser power setting ----------------------------
power = 5.43759837 # [mW]
laser.set_analog_control_mode("power")
laser.set_laser_power(power, save_memory=True)
laser.laser_on()
time.sleep(5) # allow laser current to settle
power_read = laser.get_laser_power()

assert f"{power:.0f}" == f"{power_read:.0f}"

In [10]:
laser.laser_off()

In [11]:
power = 32.90 # [mW]
laser.set_analog_control_mode("power")
laser.set_laser_power(power, save_memory=True)
# laser.laser_on()

In [13]:
# test alarm reset --------------------------------
laser.laser_off()
laser.reset_alarm()
time.sleep(10) # wait for reset
assert laser.get_status() == "Laser ON"
laser.laser_off()
assert laser.get_status() == "Standby"

In [18]:
# test TEC states -------------------------------
laser.set_TEC_enable(enable=False)
assert laser.get_TEC_state() == False
assert laser.get_status() == "Sleep"
laser.set_TEC_enable()
assert laser.get_TEC_state()

In [19]:
# test TTL digital modulation ---------------------
laser.set_analog_control_mode("power")
laser.set_digital_modulation("ExT")
assert laser.get_digital_modulation() != "ext"
laser.set_analog_control_mode("current")
laser.set_digital_modulation("ExT")
assert laser.get_digital_modulation() == "ext"
laser.set_digital_modulation("Int")
assert laser.get_digital_modulation() == 'int'
print("Test Successful!")

Failed to set command 'TTL=1'
Response from Device :'TTL=0'


Test Successful!


In [20]:
laser.close() # IMPORTANT to close the connection!!